In [2]:
from glob import glob
import json

In [3]:
files = glob('texts_8.jsonl*.splitted.requested')
files

['texts_8.jsonl01.splitted.requested',
 'texts_8.jsonl00.splitted.requested',
 'texts_8.jsonl07.splitted.requested',
 'texts_8.jsonl08.splitted.requested',
 'texts_8.jsonl03.splitted.requested',
 'texts_8.jsonl06.splitted.requested',
 'texts_8.jsonl02.splitted.requested',
 'texts_8.jsonl04.splitted.requested',
 'texts_8.jsonl05.splitted.requested']

In [4]:
mapping = {}
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue
                
            mapping[l['src']] = l['r']['result']
            
len(mapping)

888074

In [5]:
all_data = []
with open('train_8.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        data = []
        for d_ in l['data']:
            data.append({
                'value': d_,
                'value_ms': mapping.get(d_)
            })
        all_data.append(data)

In [6]:
all_data[0]

[{'value': 'March 13th saw the launch of “I Will Save You,” poems and drawings by students from all over Martha’s Vineyard.\nAll of the artwork and poems in the book were created by students from different schools around the island. The book was edited by Fan Ogilvie, educator and poet, and Richard Skidmore, Gay Head Lighthouse Keeper and poet. The book launched March 13, 2015 at Pathways in Chilmark with a poetry reading by the poets featured in the book.\nBooks will be available for purchase at island bookstores for $20 and proceeds benefit the Save the Gay Head Lighthouse project. Special thanks to Pathways Projects Institutes, Marianne Goldberg, James Weiss, Susan Stevens, Bob Moore, Amy Reece, Christine Ferrone, Todd Derry, and all the students who shared their art to help save the Light! Can you provide a summary of the "I Will Save You" book launch, including who edited the book and where it was launched?',
  'value_ms': '13 Mac menyaksikan pelancaran "Saya Akan Menyelamatkan An

In [7]:
with open('train_8.jsonl.translated', 'w') as fopen:
    for d in all_data:
        fopen.write(f'{json.dumps(d)}\n')

In [8]:
inside = []
with open('train_8.jsonl.translated') as fopen:
    for l in fopen:
        if 'model bahasa' in l:
            inside.append(l)
            if len(inside) >= 10:
                break

In [9]:
break_at = [
    'help.openai.com',
]

In [10]:
from tqdm import tqdm

all_texts = []
for i in tqdm(range(len(all_data))):
    texts = []
    for no, d in enumerate(all_data[i]):
        if d['value_ms'] is None:
            break
        if no % 2 == 0:
            t = '<manusia>: '
        else:
            t = '<bot>: '
        
        if any([b in d['value_ms'].lower() for b in break_at]):
            print(c['value_ms'], '\n-----\n')
            break
        
        n = d['value_ms']
        n = n.strip()
        if len(n) < 3:
            break
        t = t + n
        texts.append(t)

    if len(texts) % 2 != 0:
        texts = texts[:-1]
    if not len(texts):
        continue
    while len(texts) and texts[-1].startswith('<manusia>'):
        texts = texts[:-1]
    if not len(texts):
        continue
    if not texts[0].startswith('<manusia>'):
        continue
    all_texts.append('\n'.join(texts))

100%|███████████████████████████████| 150000/150000 [00:01<00:00, 111876.87it/s]


In [11]:
len(all_texts)

139392

In [12]:
for t in all_texts:
    splitted = t.split('<manusia>:')
    if '<bot>:' not in splitted[-1]:
        print(t)
        break

In [13]:
with open('part8_texts.jsonl', 'w') as fopen:
    for t in all_texts:
        fopen.write(f'{json.dumps(t)}\n')